# LCEL을 활용한 랭체인 생성하는 방법

이 노트북에서는 LangChain Expression Language (LCEL)를 사용하여 여러 컴포넌트를 연결하고 복잡한 체인을 구성하는 방법을 알아보겠습니다.

## 1. LLM 설정

먼저 기본적인 LLM을 설정합니다.


In [1]:
from langchain_ollama import ChatOllama


# Ollama를 이용한 로컬 LLM 설정
llm = ChatOllama(model="gemma3:12b")

# llm.invoke(0)


## 2. 문자열 출력 파서 사용

StrOutputParser를 사용하여 LLM의 출력을 단순 문자열로 변환할 수 있습니다.


In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 명시적인 지시사항이 포함된 프롬프트 템플릿 정의
prompt_template = PromptTemplate(
    template="What is the capital of {country}? Return the name of the city only",
    input_variables=["country"],
)
# 템플릿에 변수 값을 주입
prompt = prompt_template.invoke({"country": "France"})

print(prompt)

# LLM에 프롬프트 전달
ai_message = llm.invoke(prompt_template.invoke({"country": "France"}))

print(ai_message)

# 문자열 출력 파서 설정
output_parser = StrOutputParser()

answer = output_parser.invoke(llm.invoke(prompt_template.invoke({"country": "France"})))

text='What is the capital of France? Return the name of the city only'
content='Paris' additional_kwargs={} response_metadata={'model': 'gemma3:12b', 'created_at': '2025-03-28T11:07:44.392929Z', 'done': True, 'done_reason': 'stop', 'total_duration': 22736604208, 'load_duration': 13910955000, 'prompt_eval_count': 23, 'prompt_eval_duration': 8759364458, 'eval_count': 2, 'eval_duration': 58166417, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-86739b05-8baf-4f2d-bc25-54ae7375f010-0' usage_metadata={'input_tokens': 23, 'output_tokens': 2, 'total_tokens': 25}


## 3. LCEL을 사용한 간단한 체인 구성

파이프 연산자(|)를 사용하여 컴포넌트들을 연결합니다.


In [3]:
# 프롬프트 템플릿 -> LLM -> 출력 파서를 연결하는 체인 생성
capital_chain = prompt_template | llm | output_parser

In [4]:
# 생성된 체인 실행
capital_chain.invoke({"country": "France"})

'Paris'


## 4. 복잡한 체인 구성

여러 단계의 추론이 필요한 더 복잡한 체인을 구성해봅니다.


In [5]:
# 국가를 추측하는 프롬프트 템플릿 정의
country_prompt = PromptTemplate(
    template="""Guess the name of the country based on the following information:
    {information}
    Return the name of the country only
    """,
    input_variables=["information", "continent"],
)

# 국가 추측 체인 생성
country_chain = country_prompt | llm | output_parser
# country_chain.invoke({"information": "This country is very famous for its wine in Europe"})

In [6]:
final_chain = {"country": country_chain} | capital_chain

final_chain.invoke({"information": "This country is very famous for its wine in Europe"})

'Paris'

RunnablePassthrough를 사용한 복합 체인 구성

In [8]:
from langchain_core.runnables import RunnablePassthrough

# RunnablePassthrough를 사용하여 입력을 다음 단계로 전달하는 복합 체인 구성
final_chain = (
    {"information": RunnablePassthrough()}
    | {"country": country_chain}
    | capital_chain
)

In [9]:
# 복합 체인 실행
# 정보와 대륙을 입력하면, 해당 국가를 추측하고 그 국가의 수도를 반환
final_chain.invoke("This country is very famous for its wine in Europe")

'Paris'

복합 체인 구성 예제

In [15]:
# 국가를 추측하는 프롬프트 템플릿 정의
country_prompt = PromptTemplate(
    template="""Guess the name of the country in the {continent} based on the following information:
    {information}
    Return the name of the country only
    """,
    input_variables=["information", "continent"],
)

# 국가 추측 체인 생성
country_chain = country_prompt | llm | output_parser
# country_chain.invoke({"information": "This country is very famous for its wine in Europe"})

In [16]:
from langchain_core.runnables import RunnablePassthrough

# RunnablePassthrough를 사용하여 입력을 다음 단계로 전달하는 복합 체인 구성
final_chain = {"information": RunnablePassthrough(), "continent": RunnablePassthrough()} | {"country": country_chain} | capital_chain

In [ ]:
# 복합 체인 실행
# 정보와 대륙을 입력하면, 해당 국가를 추측하고 그 국가의 수도를 반환
final_chain.invoke( {"information": "This country is very famous for its wine in Europe", "continent": "Europe"})